In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class module(nn.Module):
    def __init__(self):
        super(module, self).__init__()
        self.layer1 = nn.Linear(4, 3) 
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        return x

model = module()
print(model)
input = torch.randn(1,4)
output = model(input)
print("Input:", input)
print("Output:", output)




module(
  (layer1): Linear(in_features=4, out_features=3, bias=True)
)
Input: tensor([[0.9017, 0.4572, 1.3625, 1.6010]])
Output: tensor([[0.0918, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)


# Q2

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
class update(nn.Module):
    def __init__(self):
        super(update, self).__init__()
        self.l1 = nn.Linear(4, 3)  
        self.l2 = nn.Linear(3, 2)  
        self.l3 = nn.Linear(2, 1)  
    
    def forward(self, x):
        x = torch.relu(self.l1(x))  
        x = torch.relu(self.l2(x)) 
        x = self.l3(x)              
        return x

model = update()
criterion = nn.MSELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01) 

input_data = torch.randn(5, 4) 
target = torch.randn(5, 1)     

output = model(input_data)
loss = criterion(output, target)
print('Initial Loss:', loss.item())
optimizer.zero_grad()   
loss.backward()        

optimizer.step()       
output = model(input_data)
new_loss = criterion(output, target)
print('New Loss :', new_loss.item())


Initial Loss: 0.09814180433750153
New Loss : 0.09792007505893707


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from ucimlrepo import fetch_ucirepo

# Step 1: Fetch the dataset
car_evaluation = fetch_ucirepo(id=19)
X = car_evaluation.data.features
y = car_evaluation.data.targets.values.flatten()  # Convert to 1D array

# Step 2: Preprocess the data
# Convert categorical variables to numeric
X = pd.get_dummies(X, drop_first=True)
y = pd.factorize(y)[0]  # Encode target labels

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X.values)
y_tensor = torch.LongTensor(y)

# Step 3: Define the model
class SimpleNN(nn.Module):
    def _init_(self):
        super(SimpleNN, self)._init_()
        self.input_layer = nn.Linear(X_tensor.shape[1], 3)  # Input layer
        self.hidden1 = nn.Linear(3, 2)  # First hidden layer
        self.hidden2 = nn.Linear(2, 1)  # Second hidden layer (Output layer)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = F.relu(self.hidden1(x))
        x = self.hidden2(x)  # Output layer
        return x

# Step 4: Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy with logits (for binary classification)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 5: Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_tensor).squeeze()  # Remove unnecessary dimensions
    loss = criterion(outputs, y_tensor.float())

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero gradients
    loss.backward()        # Backpropagation
    optimizer.step()       # Update parameters

    if (epoch + 1) % 100 == 0:  # Print every 100 epochs
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model (optional)
with torch.no_grad():
    model.eval()
    predicted = (torch.sigmoid(model(X_tensor).squeeze()) > 0.5).float()
    accuracy = (predicted == y_tensor.float()).float().mean()
    print(f'Accuracy: {accuracy:.4f}')